# This notebook is dedicated to convert the field data from another group into shapefiles. Besides, there is also a step to filter out the plots outside our RoI. 
For the field data 2, since there are several plots outside our RoI, we need to filter our them first, hence the conversion to .csv step being doable after this step

In [1]:
import pandas as pd
import geopandas as gpd

In [2]:
# Set some directories. Here we use absolute directories. 
cwd = 'c:\\Users\\m1865\\Desktop\\Ticino'
cwd_Field = cwd + '\\FieldData'
cwd_Field_Data2 = cwd_Field + '\\Field Dataset 2'
cwd_Field_Data2_CSV = cwd_Field_Data2 + "\\CSV"
cwd_Field_Data2_Shapefiles = cwd_Field_Data2 + "\\Shapefiles"

### Refine a bit the Excel file of the geographical coordinates of all the field plots and save it as .csv file

In [3]:
# Read the Excel file which contains all the geographical coordinates of the field plots! 
df_Field_Coord = pd.read_excel(cwd_Field_Data2 + "\\Dati coordinate plot forestali Parco Ticino.xlsx")
df_Field_Coord.head()

,n° plot,Y,X
0,1,45.712265,8.757804
1,2,45.702801,8.750167
2,3,45.639628,8.666780
3,4,45.630213,8.667770
4,5,45.552661,8.703548


In [4]:
# Refine a bit this dataframe and then save it as .csv file.
df_Field_Coord.columns = ['Plot','Lat','Long']
df_Field_Coord.to_csv(cwd_Field_Data2_CSV + '\\FieldData2 All LatLong.csv', index = False)

### Create a geodataframe and save it as shapefile

In [5]:
df_Field = pd.read_csv(cwd_Field_Data2_CSV + "\\FieldData2 All LatLong.csv")
# Immediately create our shapefile from this excel sheet
gdf_Field_All_4326 = gpd.GeoDataFrame(
    df_Field,
    geometry = gpd.points_from_xy(df_Field['Long'],df_Field['Lat']),
    crs = 'EPSG:4326'
)
gdf_Field_All_4326.head()

,Plot,Lat,Long,geometry
0,1,45.712265,8.757804,POINT (8.75780 45.71227)
1,2,45.702801,8.750167,POINT (8.75017 45.70280)
2,3,45.639628,8.666780,POINT (8.66678 45.63963)
3,4,45.630213,8.667770,POINT (8.66777 45.63021)
4,5,45.552661,8.703548,POINT (8.70355 45.55266)


In [6]:
gdf_Field_All_4326.to_file(cwd_Field_Data2_Shapefiles + "\\FieldData2 All LatLong.shp")

### Also create a shapefile of UTM crs (32632). 

In [7]:
gdf_Field_All_32632 = gdf_Field_All_4326[['Plot','geometry']].to_crs(32632)
gdf_Field_All_32632['UTM_X'] = gdf_Field_All_32632.geometry.x
gdf_Field_All_32632['UTM_Y'] = gdf_Field_All_32632.geometry.y
gdf_Field_All_32632 = gdf_Field_All_32632[['Plot','UTM_X','UTM_Y','geometry']]
gdf_Field_All_32632.head()

,Plot,UTM_X,UTM_Y,geometry
0,1,481149.209730,5.062108e+06,POINT (481149.210 5062107.523)
1,2,480551.550682,5.061058e+06,POINT (480551.551 5061057.811)
2,3,474030.983978,5.054063e+06,POINT (474030.984 5054062.994)
3,4,474103.856308,5.053017e+06,POINT (474103.856 5053016.615)
4,5,476860.755856,5.044390e+06,POINT (476860.756 5044389.695)


In [8]:
gdf_Field_All_32632.to_file(cwd_Field_Data2_Shapefiles + "\\FieldData2 All UTM.shp")

### Meanwhile, also save this geodataframe of UTM crs as .csv file

In [9]:
df_Field_UTM = gdf_Field_All_32632[['Plot','UTM_X','UTM_Y']]
df_Field_UTM.to_csv(cwd_Field_Data2_CSV + '\\FieldData2 All UTM.csv', index = False)

### Now we need to filter out the plots outside our RoI. 

In [10]:
# Load our region of interest shapefile! 
gdf_RoI = gpd.read_file(cwd_Field + "\\ShapefileCorretti\\confini_foreste_corretti.shp")

In [11]:
# Check which field plots (points) are within our RoI! And save their plot numbers into an array
gdf_InnerJoin = gdf_RoI.sjoin(gdf_Field_All_32632, how="inner").sort_values('Plot')
arr_ValidPlots = pd.unique(gdf_InnerJoin['Plot'])
arr_ValidPlots

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 21,
       22, 23, 24, 25, 27, 28, 29, 30, 32, 33, 34, 35, 37, 38, 39, 40],
      dtype=int64)

In [12]:
# Only keep the valid plots and recreate another new dataframe which only contains the field plots inside our RoI! 
df_Field_Valid = df_Field_Coord.copy()
mask_valid = df_Field_Valid['Plot'].isin(arr_ValidPlots)
df_Field_Valid = df_Field_Valid[mask_valid]
df_Field_Valid = df_Field_Valid.reset_index(drop = True)
df_Field_Valid.tail()

,Plot,Lat,Long
28,35,45.743313,8.627677
29,37,45.665895,8.682949
30,38,45.686724,8.756250
31,39,45.733749,8.696724
32,40,45.662013,8.737580


### Now save this dataframe as .csv file

In [13]:
df_Field_Valid_4326 = df_Field_Valid.copy()
df_Field_Valid_4326.to_csv(cwd_Field_Data2_CSV + "\\FieldData2 Valid LatLong.csv", index = False)

### Then, recreate a geodataframe from this filtered dataframe and save it as shapefile. 

In [14]:
gdf_Field_Valid_4326 = gpd.GeoDataFrame(
    df_Field_Valid_4326,
    geometry = gpd.points_from_xy(df_Field_Valid_4326['Long'],df_Field_Valid_4326['Lat']),
    crs = 'EPSG:4326'
)
gdf_Field_Valid_4326.tail()

,Plot,Lat,Long,geometry
28,35,45.743313,8.627677,POINT (8.62768 45.74331)
29,37,45.665895,8.682949,POINT (8.68295 45.66590)
30,38,45.686724,8.756250,POINT (8.75625 45.68672)
31,39,45.733749,8.696724,POINT (8.69672 45.73375)
32,40,45.662013,8.737580,POINT (8.73758 45.66201)


In [15]:
gdf_Field_Valid_4326.to_file(cwd_Field_Data2_Shapefiles + "\\FieldData2 Valid LatLong.shp")

### And again, convert the crs to UTM and save it as shapefile as well as the csv file. 

In [16]:
gdf_Field_Valid_32632 = gdf_Field_Valid_4326[['Plot','geometry']].copy().to_crs(32632)
gdf_Field_Valid_32632['UTM_X'] = gdf_Field_Valid_32632.geometry.x
gdf_Field_Valid_32632['UTM_Y'] = gdf_Field_Valid_32632.geometry.y
gdf_Field_Valid_32632 = gdf_Field_Valid_32632[['Plot','UTM_X','UTM_Y','geometry']]
gdf_Field_Valid_32632.tail()

,Plot,UTM_X,UTM_Y,geometry
28,35,471037.151278,5.065596e+06,POINT (471037.151 5065595.838)
29,37,475302.661270,5.056976e+06,POINT (475302.661 5056976.158)
30,38,481019.609917,5.059270e+06,POINT (481019.610 5059270.183)
31,39,476404.258874,5.064511e+06,POINT (476404.259 5064510.592)
32,40,479556.832180,5.056529e+06,POINT (479556.832 5056529.385)


In [17]:
gdf_Field_Valid_32632.to_file(cwd_Field_Data2_Shapefiles + "\\FieldData2 Valid UTM.shp")

In [18]:
df_Field_Valid_32632 = gdf_Field_Valid_32632[['Plot','UTM_X','UTM_Y']]
df_Field_Valid_32632.to_csv(cwd_Field_Data2_CSV + "\\FieldData2 Valid UTM.csv", index = False)